<a href="https://colab.research.google.com/github/Dworlock11/Exoplanet-Machine-Learning-Analysis/blob/main/Exoplanet_Habitability_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview**

This project intends to create machine learning models for predicting an exoplanet's type (Terran, Jovian, etc.) and mass. In addition, the most significant predictive features of these characteristics will be found and analyzed. There will be two groups of models: a classification group for predicting the type of exoplanet and a regression group for predicting the mass. Various models will be trained, tested, and evaluated, and the best model from each group will be determined. Both the performance and the time necessary for training will be considered when determining the best models.



# **Import Statements**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (make_scorer, confusion_matrix, ConfusionMatrixDisplay, classification_report, mean_absolute_error,
                             root_mean_squared_error)
from sklearn.inspection import permutation_importance
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter

# **Exploratory Data Analysis and Preprocessing**

The data from an Excel sheet is read into a DataFrame.

In [ ]:
df = pd.read_excel("Exoplanet Catalog.xlsx")

pd.set_option("display.max_columns", None)

df

Before the data can be used for model development, it needs to be cleaned and analyzed. Firstly, as many of the columns from the dataset contain a lot of null entries, it is best to simply remove the columns. All columns with more null values than a quarter of the number of rows in the dataset are removed.

In [ ]:
col_non_null_count = df.isna().sum()
cols_non_majority_null = col_non_null_count[col_non_null_count < len(df)/4].index.to_list()
df = df[cols_non_majority_null]

Additional feature selection is conducted, as many of the features are unhelpful for model training, are copies of one another, or are highly correlated.

In [ ]:
df = df.drop(["P_NAME", "P_STATUS", "P_RADIUS", "P_YEAR", "P_UPDATED", "S_NAME", "S_RADIUS", "S_ALT_NAMES", "P_HABZONE_OPT", "P_HABZONE_CON", "S_CONSTELLATION_ABR", "P_PERIOD_ERROR_MIN", "P_PERIOD_ERROR_MAX", "S_DISTANCE_ERROR_MIN", "S_DISTANCE_ERROR_MAX", "P_FLUX_MIN", "P_FLUX_MAX", "P_TEMP_EQUIL_MIN", "P_TEMP_EQUIL_MAX"], axis=1)

Categorical features with far too many unique values are removed to simplify feature encoding.

In [ ]:
# Find categorical features
cat_features = df.select_dtypes(exclude=np.number)

# Number of null values per feature
for col in cat_features.columns:
  print(col, "-", len(cat_features[col].value_counts()))

# Drop features with too many null values
df = df.drop(["S_RA_T", "S_DEC_T", "S_CONSTELLATION", "S_CONSTELLATION_ENG"], axis=1)

The data is checked for the skew of each feature to determine the appropriate imputing method for numerical data.

In [ ]:
np.abs(df.skew(axis=0, numeric_only=True, skipna=True)).sort_values(ascending=False)

Since the data is heavily skewed, the median will be chosen.

The distribution of exoplanet type is observed.

In [ ]:
df["P_TYPE"].value_counts()

A single Miniterran planet can't be split amongst a training and test set. However, if the Miniterran in the data has a radius close to that of Subterrans, it would be appropriate to mask it as one.

In [ ]:
miniterran = df[df["P_TYPE"] == "Miniterran"]
print("Miniterran mass:", f"{miniterran["P_RADIUS_EST"].iloc[0]:.3f}")

subterran = df[df["P_TYPE"] == "Subterran"]
print("Smallest Subterran mass:", f"{subterran["P_RADIUS_EST"].min():.3f}")

Indeed, the radius is around 0.33 times that of Earth, which isn't too far in value from the lightest Subterran. Therefore, the planet is masked as one.

In [ ]:
df["P_TYPE"] = df["P_TYPE"].mask(df["P_TYPE"] == "Miniterran", "Subterran")
df["P_TYPE"].value_counts()

Now, the distribution of exoplanet mass is observed.

In [ ]:
df["P_MASS_EST"].describe()

In [ ]:
plt.scatter(x=range(0, len(df.index)), y=df["P_MASS_EST"].sort_values(ascending=False))
plt.xlabel("Planet")
plt.ylabel("Mass")
plt.title("Sorted Mass of Exoplanets")
plt.show()

From this distribution, a couple of points should be made:
*   The median exoplanet seems to be approximately eight times the mass of the Earth. A minority are singificantly larger. There will almost certainly be a significant difference between the RMSE and the MAE when evaluating the models, since RMSE is more sensitive to outliers.
*   Because the smallest planets and the largest are orders of magnitude apart, it would make sense to tranform the mass into log space. This tranformed feature will be stored in a copy of the dataset.
*   Huber loss will be used for hyperparameter training and permutation in order to balance the majority of small planets with the minority of massive ones. A manual definition is created, since no built-in one is available in the version of sci-kit learn used in this project.
*   It's not clear what exactly it means for a planet to have a mass of 0.0. It might be a mistake. Such entries will be removed to be safe.

In [ ]:
# Define Huber loss
def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    abs_error = np.abs(error)

    quadratic = np.minimum(abs_error, delta)
    linear = abs_error - quadratic

    return np.mean(0.5 * quadratic**2 + delta * linear)

# Create scorer
huber_scorer = make_scorer(huber_loss, greater_is_better=False, delta=1.0)

# Remove entries with 0 masses
df = df[df["P_MASS_EST"] != 0.0]

# Copy data and transform mass into logspace
log_df = df.copy()
log_df["Log_Mass"] = np.log10(log_df["P_MASS_EST"])
log_df = log_df.drop("P_MASS_EST", axis=1)

All rows where the target value is null, if any, are removed to prevent errors.

In [ ]:
# Remove null entries in the type column and display number of null entries
print("Number of null values in type column:", df["P_TYPE"].isna().sum())
type_na = df[df["P_TYPE"].isna()].index
df = df.drop(type_na)
print("Number of null values after removal:", df["P_TYPE"].isna().sum(), "\n")

# Remove null entries in the mass column and display number of null entries
print("Number of null values in mass column:", log_df["Log_Mass"].isna().sum())
mass_na = log_df[log_df["P_TYPE"].isna()].index
log_df = log_df.drop(mass_na)
print("Number of null values after removal:", log_df["P_TYPE"].isna().sum())

Now, model development can begin.

# **Exoplanet Type Classification**

The classification models will be trained first, starting with logistic regression.

## Logistic Regression

The data is separated into the features and the target.

In [ ]:
X = df.drop("P_TYPE", axis=1)
y = df["P_TYPE"]

The data is split into the training and testing data. It is stratified by the exoplanet type to make sure that a proportional number of each type is present in both the training set and the test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=9)

The data preprocessor is created, including data imputing, standardizing, and encoding.

In [ ]:
# Find numerical and categorical columns
num_features = X_train.select_dtypes(include=np.number)
cat_features = X_train.select_dtypes(exclude=np.number)
num_col_names = num_features.columns
cat_col_names = cat_features.columns

# Create transformer for numerical columns
num_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Create transformer for categorical columns
linear_cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", feature_name_combiner="concat"))
])

# Combine transformers
log_preprocessor = ColumnTransformer([
    ("num_transformer", num_transformer, num_col_names),
    ("linear_cat_transformer", linear_cat_transformer, cat_col_names)
])

The pipeline is created and the hyperparameter C is tuned to prevent overfitting.

In [ ]:
log_pipe = Pipeline([
    ("log_preprocessor", log_preprocessor),
    ("log_reg", LogisticRegression(
        solver="lbfgs",
        penalty="l2",
        max_iter=300
    ))
])

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)

param_dist = {
    "log_reg__C": np.logspace(-3, 3, 15),
}

search = RandomizedSearchCV(log_pipe, param_distributions=param_dist, n_iter=10, cv=kf, random_state=9, n_jobs=-1)

The model is now trained and the tuned values for each hyperparameter are displayed.

In [ ]:
# Supress convergence warnings
simplefilter("ignore", category=ConvergenceWarning)

search.fit(X_train, y_train)

best_model = search.best_estimator_

print("C:", f"{search.best_params_["log_reg__C"]:.3f}")

The model is tested and evaluated.

In [ ]:
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

The model performs very well overall. Interestingly, Subterrans performed significantly worse than the other types, likely due to a much smaller number of entries in the dataset. A confusion matrix illustrates these findings.

In [ ]:
class_labels = ["Jovian", "Neptunian", "Subterran", "Superterran", "Terran"]

cm = confusion_matrix(y_test, y_pred, labels=class_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Logistic Regression Confusion Matrix")
plt.xticks(rotation=45)
plt.show()

Permutation is used to find the importance of the individual features. It will be used across all models for standardized results. The test set must be manually transformed with all preprocessing steps before implementing permutation to match the number of columns present after feature encoding.

In [ ]:
# Extract components
preprocessor = best_model.named_steps["log_preprocessor"]
log_reg = best_model.named_steps["log_reg"]

raw_feature_names = preprocessor.get_feature_names_out()

# Remove transformer names from features
clean_feature_names = [
    name.split("__", 1)[1]
    if "__" in name else name
    for name in raw_feature_names
]

# Transform X_test into expanded feature space
X_test_transformed = preprocessor.transform(X_test)

# Run permutation importance
importances = permutation_importance(log_reg, X_test_transformed, y_test, n_repeats=10, random_state=9, n_jobs=-1)

# Display results
highest_importances = pd.Series(importances.importances_mean, index=clean_feature_names).sort_values(ascending=False).head(10)

plt.bar(highest_importances.index, highest_importances)
plt.xticks(rotation=70)
plt.title("Logistic Regression Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Drop in Performance")
plt.show()

Apparently, the most important feature for predicting the type of the planet is its radius. This make sense, as Jovian planets are significantly larger in size than Terrans, for example.

## Polynomial Logistic Regression

Now, polynomial features are added to see if they will make a significant difference.

A new preprocessor is created to accomodate polynomial features.

In [ ]:
# Build polynomial transformer
poly_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler())
])

# Combine transformers
poly_log_preprocessor = ColumnTransformer([
    ("poly_transformer", poly_transformer, num_col_names),
    ("linear_cat_transformer", linear_cat_transformer, cat_col_names)
])

The pipeline is created and hyperparameter tuning is implemented.

In [ ]:
poly_log_pipe = Pipeline([
    ("poly_log_preprocessor", poly_log_preprocessor),
    ("log_reg", LogisticRegression(
        solver="lbfgs",
        penalty="l2",
        max_iter=300
    ))
])

param_dist = {
    "log_reg__C": np.logspace(-3, 3, 15),
}

search = RandomizedSearchCV(poly_log_pipe, param_distributions=param_dist, n_iter=10, cv=kf, random_state=9, n_jobs=-1)

The model is again trained and optimal regularization strength is shown.

In [ ]:
search.fit(X_train, y_train)

best_model = search.best_estimator_

print("C:", f"{search.best_params_["log_reg__C"]:.3f}")

The model is tested and evaluated.

In [ ]:
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=class_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Polynomial Logistic Regression Confusion Matrix")
plt.xticks(rotation=45)
plt.show()

The model performs around the same as without polynomial features. However, the time necessary to train is significantly longer. Therefore, there seems to be little reason to include polynomial features.

Feature importance is ignored, as most of the features are simply engineered polynomial features, giving little legitimate insight.

## Decision Tree

Now, a decision tree model will be trained following the same process.

A new categorical transformer is created using ordinal encoding, which is suitable for tree-based models and better than one-hot encoding, since it doesn't create many sparse features.

In [ ]:
# Create categorical transformer for tree models
tree_cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])

# Combine transformers
tree_preprocessor = ColumnTransformer([
    ("num_transformer", num_transformer, num_col_names),
    ("tree_cat_transformer", tree_cat_transformer, cat_col_names)
])

The pipeline is created and hyperparameter tuning is implemented, testing values for the major hyperparameters of decision trees.

In [ ]:
tree_pipe = Pipeline([
    ("tree_preprocessor", tree_preprocessor),
    ("dec_tree", DecisionTreeClassifier())
])

param_dist = {
    "dec_tree__max_depth": [None, 2, 5, 10, 20],
    "dec_tree__min_samples_split": [2, 5, 10, 20, 50],
    "dec_tree__min_samples_leaf": [1, 2, 5, 10, 20],
    "dec_tree__max_features": ["sqrt", "log2", None],
}

search = RandomizedSearchCV(tree_pipe, param_distributions=param_dist, n_iter=10, cv=kf, random_state=9, n_jobs=-1)

The model is trained and optimized hyperparameters are shown.

In [ ]:
search.fit(X_train, y_train)

best_model = search.best_estimator_

for param, value in search.best_params_.items():
  print(param.split("__", 1)[1], ":" , value)

The model is tested and evaluated.

In [ ]:
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=class_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Decision Tree Confusion Matrix")
plt.xticks(rotation=45)
plt.show()

The metrics are notably better than those from the logistic regression model, especially for Subterrans. Perhaps decision trees are better suited for multiclass classification, even with little data.

Permutation is once again used to discover feature importance.

In [ ]:
# Extract components
preprocessor = best_model.named_steps["tree_preprocessor"]
dec_tree = best_model.named_steps["dec_tree"]

# Remove name of transformer from each feature
raw_feature_names = preprocessor.get_feature_names_out()
clean_feature_names = [
    name.split("__", 1)[1] if "__" in name else name
    for name in raw_feature_names
]

# Transform X_test into expanded feature space
X_test_transformed = preprocessor.transform(X_test)

# Calculate importances
importances = permutation_importance(dec_tree, X_test_transformed, y_test, n_repeats=10, random_state=9, n_jobs=-1)

# Display results
highest_importances = pd.Series(importances.importances_mean, index=clean_feature_names).sort_values(ascending=False).head(10)

plt.bar(highest_importances.index, highest_importances)
plt.xticks(rotation=70)
plt.title("Decision Tree Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Drop in Performance")
plt.show()

In comparison to the logistic regression model, mass in a decision tree is a more significant predictor of exoplanet type. This may be because the logistic regression models found it more difficult to determine the importance of mass due to the nonlinear nature of the feature.

## Random Forest

Now, a random forest model will be trained.

The pipeline is created and hyperparameter tuning is implemented, testing ranges of values for the major hyperparameters.

In [ ]:
forest_pipe = Pipeline([
    ("tree_preprocessor", tree_preprocessor),
    ("rand_for", RandomForestClassifier())
])

param_dist = {
    "rand_for__n_estimators": [200, 400, 600, 800],
    "rand_for__max_depth": [None, 5, 10, 20, 40],
    "rand_for__min_samples_split": [2, 5, 10, 20],
    "rand_for__min_samples_leaf": [1, 2, 5, 10],
    "rand_for__max_features": ["sqrt", "log2", None],
    "rand_for__bootstrap": [True, False],
}

search = RandomizedSearchCV(forest_pipe, param_distributions=param_dist, n_iter=10, cv=kf, random_state=9, n_jobs=-1)

The model is trained and hyperparameter values are shown.

In [ ]:
search.fit(X_train, y_train)

best_model = search.best_estimator_

for param, value in search.best_params_.items():
  print(param.split("__", 1)[1], ":" , value)

The model is tested and evaluated.

In [ ]:
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=class_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Random Forest Confusion Matrix")
plt.xticks(rotation=45)
plt.show()

The metrics are around the same as those for the decision tree model. However, it takes much longer to fit, making random forests apparently unnecessary for this task.

Permutation is once again used to discover feature importance.

In [ ]:
# Extract components
preprocessor = best_model.named_steps["tree_preprocessor"]
rand_for = best_model.named_steps["rand_for"]

# Run permutation importance
importances = permutation_importance(rand_for, X_test_transformed, y_test, n_repeats=10, random_state=9, n_jobs=-1)

# Display results
highest_importances = pd.Series(importances.importances_mean, index=clean_feature_names).sort_values(ascending=False).head(10)

plt.bar(highest_importances.index, highest_importances)
plt.xticks(rotation=70)
plt.title("Random Forest Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Drop in Performance")
plt.show()

The results are similar to those from the decision tree model.

This concludes the development of the classfication models.

# **Exoplanet Mass Prediction**

Now, regression models will be created to predict exoplanet mass.

## Ridge Regression

A simple linear regression model is a good starting point. Specifically, Ridge will be chosen over standard linear regression to be able to use regularization.

The data is split into the features and the target.

In [ ]:
X = log_df.drop("Log_Mass", axis=1)
y = log_df["Log_Mass"]

The data is split into the training and testing data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

The transformers used for the logistic regression model can be reused.

In [ ]:
# Separate numerical and categorical features
num_features = X_train.select_dtypes(include=np.number)
cat_features = X_train.select_dtypes(exclude=np.number)
num_col_names = num_features.columns
cat_col_names = cat_features.columns

# Combine transformers
ridge_preprocessor = ColumnTransformer([
    ("num_transformer", num_transformer, num_col_names),
    ("linear_cat_transformer", linear_cat_transformer, cat_col_names)
])

The pipeline is created and hyperparameter tuning is implemented.

In [ ]:
ridge_pipe = Pipeline([
    ("ridge_preprocessor", ridge_preprocessor),
    ("ridge", Ridge())
])

kf = KFold(n_splits=5, shuffle=True, random_state=9)

param_dist = {
    "ridge__alpha": np.logspace(-4, 4)
}

search = RandomizedSearchCV(ridge_pipe, param_distributions=param_dist, scoring=huber_scorer, n_iter=10, cv=kf,
                            random_state=9, n_jobs=-1)

The model is trained and the optimized alpha value is displayed.

In [ ]:
search.fit(X_train, y_train)

best_model = search.best_estimator_

print("alpha:", f"{search.best_params_["ridge__alpha"]:.3f}")

RMSE and MAE are used to evaluate the model.

In [ ]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"MAE factor  : {10**mae:.2f}×")
print(f"RMSE factor : {10**rmse:.2f}×")

The results aren't promising. To understand why, the array of predicted values is analyzed along with the true values.

In [ ]:
y_pred_se = pd.Series(y_pred)

y_pred_se.describe()

In [ ]:
y_test_se = y_test.reset_index(drop=True)

y_test_se.describe()

There appears to be a major outlier present in the predicted values. This is further visible in a scatter plot of the actual and predicted values.

In [ ]:
plt.scatter(x=range(0, len(y_test)), y=10**(y_test.reset_index(drop=True)), color="blue", alpha=0.4, label="Actual")
plt.scatter(x=range(0, len(y_pred)), y=10**y_pred, color="green", alpha=0.4, label="Predicted")
plt.yscale("log")
plt.legend()
plt.xlabel("Planet")
plt.ylabel("Mass")
plt.title("Actual vs Predicted Mass")
plt.show()

It appears that one heavily inacurate prediction is causing the metrics to significantly worsen. Therefore, linear regression doesn't seem to be a good model for this task.

## Polynomial Ridge Regression

Perhaps adding polynomial features will improve the results.

A polynomial transformer is created.

In [ ]:
# Create polynomial transformer
poly_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("poly", PolynomialFeatures(include_bias=False)),
    ("scaler", StandardScaler())
])

# Combine transformers
poly_ridge_preprocessor = ColumnTransformer([
    ("poly_transformer", poly_transformer, num_col_names),
    ("linear_cat_transformer", linear_cat_transformer, cat_col_names)
])

The pipeline is created and the alpha value along with the polynomial degree are tuned.

In [ ]:
poly_ridge_pipe = Pipeline([
    ("poly_ridge_preprocessor", poly_ridge_preprocessor),
    ("ridge", Ridge())
])

param_dist = {
    "poly_ridge_preprocessor__poly_transformer__poly__degree" : [2, 3],
    "ridge__alpha" : np.logspace(-4, 4)
}

search = RandomizedSearchCV(poly_ridge_pipe, param_distributions=param_dist, scoring=huber_scorer, n_iter=10, cv=kf,
                            random_state=9, n_jobs=-1)

The model is trained and the hyperparameter values are shown.

In [ ]:
search.fit(X_train, y_train)

best_model = search.best_estimator_

print("alpha:", f"{search.best_params_["ridge__alpha"]:.3f}")
print("Degree:", search.best_params_["poly_ridge_preprocessor__poly_transformer__poly__degree"])

RMSE and MAE are used to evaluate the model.

In [ ]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"MAE factor  : {10**mae:.2f}×")
print(f"RMSE factor : {10**rmse:.2f}×")

The results are even worse than without polynomial features. In fact, the model is almost comically bad. The predicted values are again analyzed to learn why.

In [ ]:
y_pred_se = pd.Series(y_pred)

y_pred_se.describe()

In [ ]:
y_test_se.describe()

In [ ]:
plt.scatter(x=range(0, len(y_test)), y=10**y_test.reset_index(drop=True), color="blue", alpha=0.4, label="Actual")
plt.scatter(x=range(0, len(y_pred)), y=10**y_pred, color="green", alpha=0.4, label="Predicted")
plt.yscale("log")
plt.legend()
plt.xlabel("Planet")
plt.ylabel("Mass")
plt.title("Actual vs Predicted Mass")
plt.show()

As before, there is a single inacurate value bringing the model down. With or without polynomial features, linear regression is inadequate.

## Decision Tree Regressor

Now, a decision tree is created, hopefully bearing better results.

A suitable pipeline is created and important hyperparameters are tuned.

In [ ]:
# Combine numerical and categorical transformers
tree_preprocessor = ColumnTransformer([
    ("num_transformer", num_transformer, num_col_names),
    ("tree_cat_transformer", tree_cat_transformer, cat_col_names)
])

tree_pipe = Pipeline([
    ("tree_preprocessor", tree_preprocessor),
    ("dec_tree", DecisionTreeRegressor())
])

param_dist = {
    "dec_tree__max_depth": [None, 3, 5, 10, 20],
    "dec_tree__min_samples_split": [2, 5, 10, 20, 50],
    "dec_tree__min_samples_leaf": [1, 2, 5, 10, 20, 50],
    "dec_tree__max_features": [None, "sqrt", "log2"]
}

search = RandomizedSearchCV(tree_pipe, param_distributions=param_dist, scoring=huber_scorer, n_iter=10, cv=kf, random_state=9, n_jobs=-1)

The model is trained and hyperparameter values are displayed.

In [ ]:
search.fit(X_train, y_train)

best_model = search.best_estimator_

for param, value in search.best_params_.items():
  print(param.split("__", 1)[1], ":" , value)

As before, RMSE and MAE are used to evaluate the model.

In [ ]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"MAE factor  : {10**mae:.2f}×")
print(f"RMSE factor : {10**rmse:.2f}×")

The results point to a strong model. Real exoplanet masses often have 30-100% observational uncertainty, so a 24% error factor is very good. This is also evident in a scatter plot.

In [ ]:
plt.scatter(x=range(0, len(y_test)), y=10**(y_test.reset_index(drop=True)), color="blue", alpha=0.4, label="Actual")
plt.scatter(x=range(0, len(y_pred)), y=10**y_pred, color="green", alpha=0.4, label="Predicted")
plt.yscale("log")
plt.legend()
plt.xlabel("Planet")
plt.ylabel("Mass")
plt.title("Actual vs Predicted Mass")
plt.show()

Permutation is used to find important features.

In [ ]:
# Extract components
preprocessor = best_model.named_steps["tree_preprocessor"]
dec_tree = best_model.named_steps["dec_tree"]

# Remove name of transformer from each feature
raw_feature_names = preprocessor.get_feature_names_out()
clean_feature_names = [
    name.split("__", 1)[1] if "__" in name else name
    for name in raw_feature_names
]

# Transform X_test into expanded feature space
X_test_transformed = preprocessor.transform(X_test)

# Run permutation importance on the classifier only
importances = permutation_importance(dec_tree, X_test_transformed, y_test, scoring=huber_scorer, n_repeats=10,
                                     random_state=9, n_jobs=-1)

# Display results
highest_importances = pd.Series(importances.importances_mean, index=clean_feature_names).sort_values(ascending=False).head(10)

plt.bar(highest_importances.index, highest_importances)
plt.xticks(rotation=70)
plt.title("Decision Tree Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Drop in Performance")
plt.show()

By far the most important features are the planet's type and radius. These findings make sense; mass and radius are often correlated and a Jovian planet will certainly have a greater mass than a Terran one.

## Random Forest Regressor

Perhaps a random forest will be even better for prediction.

As usual, the pipeline is created and hyperparameter tuning is implemented.

In [ ]:
tree_pipe = Pipeline([
    ("tree_preprocessor", tree_preprocessor),
    ("rand_for", RandomForestRegressor())
])

param_dist = {
    "rand_for__n_estimators": [200, 300, 500, 800],
    "rand_for__max_depth": [None, 5, 10, 20, 40],
    "rand_for__min_samples_split": [2, 5, 10, 20],
    "rand_for__min_samples_leaf": [1, 2, 5, 10, 20],
    "rand_for__max_features": ["sqrt", "log2", None],
    "rand_for__bootstrap": [True, False]
}

search = RandomizedSearchCV(tree_pipe, param_distributions=param_dist, scoring=huber_scorer, n_iter=10, cv=kf, random_state=9, n_jobs=-1)

Again, the model is trained and tuned hyperparameter values are shown.

In [ ]:
search.fit(X_train, y_train)

best_model = search.best_estimator_

for param, value in search.best_params_.items():
  print(param.split("__", 1)[1], ":" , value)

RMSE and MAE are used to evaluate the model as usual.

In [ ]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"MAE factor  : {10**mae:.2f}×")
print(f"RMSE factor : {10**rmse:.2f}×")

The model performs slightly better than the decision tree model, but takes much more time to train. These results are similar to those from the classification tree models.

A plot is generated as before to underscore these findings.

In [ ]:
plt.scatter(x=range(0, len(y_test)), y=10**(y_test.reset_index(drop=True)), color="blue", alpha=0.4, label="Actual")
plt.scatter(x=range(0, len(y_pred)), y=10**y_pred, color="green", alpha=0.4, label="Predicted")
plt.yscale("log")
plt.legend()
plt.xlabel("Planet")
plt.ylabel("Mass")
plt.title("Actual vs Predicted Mass")
plt.show()

The predicted and actual values are relatively close together, implying little erorr in prediction.

Feature importance is analyzed.

In [ ]:
# Extract components
rand_for = best_model.named_steps["rand_for"]

# Run permutation importance on the classifier only
importances = permutation_importance(rand_for, X_test_transformed, y_test, scoring=huber_scorer, n_repeats=10,
                                     random_state=9, n_jobs=-1)
# Display results
highest_importances = pd.Series(importances.importances_mean, index=clean_feature_names).sort_values(ascending=False).head(10)

plt.bar(highest_importances.index, highest_importances)
plt.xticks(rotation=70)
plt.title("Random Forest Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Drop in Performance")
plt.show()

The results are similar to those from the decision tree model.

# **Conclusion**

To conclude, significant insights can be drawn from this analysis. Firstly, decision trees seem to be the all-around best models for both classification of exoplanet type and of mass. Random forests produce slightly better results but take longer to train. The tree models may have outperformed the linear models due to not being restricted to following a linear structure. This was especially important for predicting exoplanet mass, which doesn't follow a linear pattern. Another important discovery is that the most important features for classifying exoplanets were their radius and mass. The most important features for predicting a planet's mass were its type and radius. Therefore, exoplanet type, radius, and mass seem to be highly correlated. This has scientific precedent, as Jovian and Neptunian planets are known to have greater masses and sizes than Subterrans, for example. One can presume that these are important characteristics used by scientists when creating models with exoplanet data.

# **Copyright**

The data used in this project was taken from the exoplanet catalog found here: https://www.kaggle.com/datasets/chandrimad31/phl-exoplanet-catalog?resource=download.

I claim no ownership of the data. All rights reserved to the rightful owners.